In [1]:
%matplotlib inline
import math
import pandas as pd
from tsmom_model import *
import numpy as np
from datetime import datetime
import PPCA_MR as p
import statsmodels.api as sm

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=2)
pylab.rcParams['figure.figsize'] = (24, 18)

# Hand cleansed
Hand cleaned to removed duplicates

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


# Amihud methodology

Amihud (2002) developed the illiquidity measure expressed by:

$$ Amihud =\frac{1}{N} \sum \frac{|r_t|}{\text{dvol}_t} $$

where $r_t$ is the return on day $t$ and $Volume_t$ is the dollar volume on day $t$. 

The benefit of taking the daily return is to weight the move by change of price and the volume.


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

# Academic papers
Based on Illin, A., Raiko, T. 2010 paper called "Practical Approaches to Principal Component Analysis in the Presence of Missing Values"

Tipping and Bishop (1999) introducted probablistic formulation of PCA (PPCA)


Use a flavor of Probabilistic PCA that is robust to missing data (see Ilin and Raiko 2010). Rather than the vanilla one-shot SVD, PPCA uses an iterative EM procedure/fixed point algorithm. From an initial guess, it’ll alternatively interpolate missing data and update the components until convergence.

On a side note, PPCA interpolates using information from all series, making it a multivariate interpolator. Be careful though, fitting the model to too many components will likely lead to overfitting problems in the interpolated data.

Also note that I am using the standardization that is mentioned in Korajecyk and Sadka (2008) of expanding zscores


In [5]:
#data=calc_zscore_ew(amihud.dropna(how='all')).dropna(how='all')
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')

tsmom_model.py:399: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
tsmom_model.py:399: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


In [6]:
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

In [7]:
sector_PC={}
sector_variance_explained=pd.DataFrame()

no_pc=3
for sect in d_map.keys():
    try: 
        PPCA=p.PPCA()
        PPCA.fit(np.array(sector_zscores[sect]),d=no_pc)
        sector_variance_explained[sect] = PPCA.var_exp
        pcs=pd.DataFrame()
        for i in range(0,no_pc,1):
            x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
            x.set_axis(sector_zscores[sect].index)
            pcs[i]=x
        sector_PC[sect] = pcs
    except:
        print sect

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  del sys.path[0]


In [8]:
no_fx = [i for i in data.columns if i not in d_map['Currencies']]
PPCA=p.PPCA()
PPCA.fit(np.array(data[no_fx]),d=no_pc)
sector_variance_explained[sect] = PPCA.var_exp
pcs=pd.DataFrame()
for i in range(0,no_pc,1):
    x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
    x.set_axis(sector_zscores[sect].index)
    pcs[i]=x
sector_PC['Total'] = pcs

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  


In [9]:
totalFactor=pd.DataFrame()
for s in ['Agriculturals',
 'Energies',
 'Equities',
 'Metals',
 'Fixed Income',
 'Total']:
    totalFactor[s]=sector_PC[s].mean(axis=1)

# Conditional correlation on crisis



In [10]:
vix_full=pd.read_csv('vix.csv',parse_dates=['date'],index_col=0)

In [37]:
vix=vix_full['2000':'2016'].VIX

In [38]:
# Above median vix
ind=vix[vix>vix.median()].index
totalFactor.ix[ind].corr().round(2).abs()

,Agriculturals,Energies,Equities,Metals,Fixed Income,Total
Agriculturals,1.00,0.25,0.35,0.59,0.32,0.54
Energies,0.25,1.00,0.24,0.05,0.20,0.26
Equities,0.35,0.24,1.00,0.51,0.46,0.68
Metals,0.59,0.05,0.51,1.00,0.43,0.63
Fixed Income,0.32,0.20,0.46,0.43,1.00,0.34
Total,0.54,0.26,0.68,0.63,0.34,1.00


In [39]:
print totalFactor.ix[ind].corr().round(2).abs().to_latex()

\begin{tabular}{lrrrrrr}
\toprule
{} &  Agriculturals &  Energies &  Equities &  Metals &  Fixed Income &  Total \\
\midrule
Agriculturals &           1.00 &      0.25 &      0.35 &    0.59 &          0.32 &   0.54 \\
Energies      &           0.25 &      1.00 &      0.24 &    0.05 &          0.20 &   0.26 \\
Equities      &           0.35 &      0.24 &      1.00 &    0.51 &          0.46 &   0.68 \\
Metals        &           0.59 &      0.05 &      0.51 &    1.00 &          0.43 &   0.63 \\
Fixed Income  &           0.32 &      0.20 &      0.46 &    0.43 &          1.00 &   0.34 \\
Total         &           0.54 &      0.26 &      0.68 &    0.63 &          0.34 &   1.00 \\
\bottomrule
\end{tabular}



In [40]:
# Below median vix
ind=vix[vix<=vix.median()].index
totalFactor.ix[ind].corr().round(2).abs()

,Agriculturals,Energies,Equities,Metals,Fixed Income,Total
Agriculturals,1.00,0.39,0.26,0.05,0.30,0.12
Energies,0.39,1.00,0.13,0.07,0.41,0.14
Equities,0.26,0.13,1.00,0.29,0.05,0.80
Metals,0.05,0.07,0.29,1.00,0.02,0.03
Fixed Income,0.30,0.41,0.05,0.02,1.00,0.05
Total,0.12,0.14,0.80,0.03,0.05,1.00


In [41]:
print totalFactor.ix[ind].corr().round(2).abs().to_latex()

\begin{tabular}{lrrrrrr}
\toprule
{} &  Agriculturals &  Energies &  Equities &  Metals &  Fixed Income &  Total \\
\midrule
Agriculturals &           1.00 &      0.39 &      0.26 &    0.05 &          0.30 &   0.12 \\
Energies      &           0.39 &      1.00 &      0.13 &    0.07 &          0.41 &   0.14 \\
Equities      &           0.26 &      0.13 &      1.00 &    0.29 &          0.05 &   0.80 \\
Metals        &           0.05 &      0.07 &      0.29 &    1.00 &          0.02 &   0.03 \\
Fixed Income  &           0.30 &      0.41 &      0.05 &    0.02 &          1.00 &   0.05 \\
Total         &           0.12 &      0.14 &      0.80 &    0.03 &          0.05 &   1.00 \\
\bottomrule
\end{tabular}



# FHT method

In [42]:
FHT=calculate_FHT(cleansed)

tsmom_model.py:434: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:435: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:436: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:436: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [43]:
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')

In [44]:
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

In [45]:
sector_PC={}
sector_variance_explained=pd.DataFrame()

no_pc=3
for sect in d_map.keys():
    try: 
        PPCA=p.PPCA()
        PPCA.fit(np.array(sector_zscores[sect]),d=no_pc)
        sector_variance_explained[sect] = PPCA.var_exp
        pcs=pd.DataFrame()
        for i in range(0,no_pc,1):
            x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
            x.set_axis(sector_zscores[sect].index)
            pcs[i]=x
        sector_PC[sect] = pcs
    except:
        print sect

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  del sys.path[0]


In [46]:
PPCA=p.PPCA()
PPCA.fit(np.array(data),d=no_pc)
sector_variance_explained[sect] = PPCA.var_exp
pcs=pd.DataFrame()
for i in range(0,no_pc,1):
    x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
    x.set_axis(sector_zscores[sect].index)
    pcs[i]=x
sector_PC['Total'] = pcs


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: set_axis currently defaults to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  import sys


In [47]:
totalFactor_FHT=pd.DataFrame()
for s in ['Agriculturals','Currencies',
 'Energies',
 'Equities',
 'Metals',
 'Fixed Income',
 'Total']:
    totalFactor_FHT[s]=sector_PC[s].mean(axis=1)

In [48]:
# Above median vix
ind=vix[vix>vix.median()].index
totalFactor_FHT.ix[ind].corr().round(2).abs()

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,Agriculturals,Currencies,Energies,Equities,Metals,Fixed Income,Total
Agriculturals,1.00,0.24,0.25,0.37,0.49,0.52,0.26
Currencies,0.24,1.00,0.37,0.33,0.34,0.33,0.37
Energies,0.25,0.37,1.00,0.37,0.30,0.32,0.43
Equities,0.37,0.33,0.37,1.00,0.52,0.43,0.93
Metals,0.49,0.34,0.30,0.52,1.00,0.45,0.56
Fixed Income,0.52,0.33,0.32,0.43,0.45,1.00,0.41
Total,0.26,0.37,0.43,0.93,0.56,0.41,1.00


In [49]:
print totalFactor_FHT.ix[ind].corr().round(2).abs().to_latex()

\begin{tabular}{lrrrrrrr}
\toprule
{} &  Agriculturals &  Currencies &  Energies &  Equities &  Metals &  Fixed Income &  Total \\
\midrule
Agriculturals &           1.00 &        0.24 &      0.25 &      0.37 &    0.49 &          0.52 &   0.26 \\
Currencies    &           0.24 &        1.00 &      0.37 &      0.33 &    0.34 &          0.33 &   0.37 \\
Energies      &           0.25 &        0.37 &      1.00 &      0.37 &    0.30 &          0.32 &   0.43 \\
Equities      &           0.37 &        0.33 &      0.37 &      1.00 &    0.52 &          0.43 &   0.93 \\
Metals        &           0.49 &        0.34 &      0.30 &      0.52 &    1.00 &          0.45 &   0.56 \\
Fixed Income  &           0.52 &        0.33 &      0.32 &      0.43 &    0.45 &          1.00 &   0.41 \\
Total         &           0.26 &        0.37 &      0.43 &      0.93 &    0.56 &          0.41 &   1.00 \\
\bottomrule
\end{tabular}



In [50]:
# Below median vix
ind=vix[vix<=vix.median()].index
totalFactor_FHT.ix[ind].corr().round(2).abs()

,Agriculturals,Currencies,Energies,Equities,Metals,Fixed Income,Total
Agriculturals,1.00,0.18,0.12,0.03,0.03,0.17,0.16
Currencies,0.18,1.00,0.25,0.16,0.11,0.38,0.06
Energies,0.12,0.25,1.00,0.05,0.04,0.16,0.11
Equities,0.03,0.16,0.05,1.00,0.34,0.32,0.88
Metals,0.03,0.11,0.04,0.34,1.00,0.08,0.28
Fixed Income,0.17,0.38,0.16,0.32,0.08,1.00,0.34
Total,0.16,0.06,0.11,0.88,0.28,0.34,1.00


In [51]:
print totalFactor_FHT.ix[ind].corr().round(2).abs().to_latex()

\begin{tabular}{lrrrrrrr}
\toprule
{} &  Agriculturals &  Currencies &  Energies &  Equities &  Metals &  Fixed Income &  Total \\
\midrule
Agriculturals &           1.00 &        0.18 &      0.12 &      0.03 &    0.03 &          0.17 &   0.16 \\
Currencies    &           0.18 &        1.00 &      0.25 &      0.16 &    0.11 &          0.38 &   0.06 \\
Energies      &           0.12 &        0.25 &      1.00 &      0.05 &    0.04 &          0.16 &   0.11 \\
Equities      &           0.03 &        0.16 &      0.05 &      1.00 &    0.34 &          0.32 &   0.88 \\
Metals        &           0.03 &        0.11 &      0.04 &      0.34 &    1.00 &          0.08 &   0.28 \\
Fixed Income  &           0.17 &        0.38 &      0.16 &      0.32 &    0.08 &          1.00 &   0.34 \\
Total         &           0.16 &        0.06 &      0.11 &      0.88 &    0.28 &          0.34 &   1.00 \\
\bottomrule
\end{tabular}

